In [ ]:
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from rasterio.enums import Resampling
from PIL import ImageColor
from skimage.exposure import rescale_intensity
import rasterio as rio
import os
import pandas as pd
from PIL import ImageColor
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, Conv2DTranspose, Dropout, MaxPooling2D, Input, concatenate, Concatenate
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from skimage.exposure import rescale_intensity
from rasterio.enums import Resampling
import rasterio.warp as warp

import torch

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import MeanIoU, CategoricalAccuracy, Precision, Recall, AUC


In [ ]:
#Localización del data: JSON de clases con paleta de colores, imagen original y su máscara.
colores_dir = r'C:\Users\Usuari\Documents\TFM_Codigos\MODELO\data/7Labels.json'
colores = json.load(open(colores_dir))
colores_df = pd.DataFrame(colores)
colores_df["values_normalize"] = colores_df.index
colores_df["palette"] = "#" + colores_df["palette"]
values = colores_df["values"].to_list()
values_norm = colores_df["values_normalize"].to_list()
palette = colores_df["palette"].to_list()
labels = colores_df["label"].to_list()
dict_values = {}
dict_label = {}
dict_palette = {}
dict_palette_hex = {}
for x in range(0, len(values)):
    dict_values[values[x]] = values_norm[x]
    dict_label[values_norm[x]] = labels[x]
    dict_palette[values_norm[x]] = ImageColor.getrgb(palette[x])
    dict_palette_hex[values_norm[x]] = palette[x]
cmap = ListedColormap(palette)
patches = [
    mpatches.Patch(color=palette[i], label=labels[i]) for i in range(len(values))
]
legend = {
    "handles": patches,
    "bbox_to_anchor": (1.05, 1),
    "loc": 2,
    "borderaxespad": 0.0,
}
colores_df

In [ ]:
import numpy as np
import os
import re

def load_and_combine_datasets(folder_path):
    files = os.listdir(folder_path)
    image_train_pattern = re.compile(r".npy") 
    image_test_pattern = re.compile(r".npy")
    mascara_train_pattern = re.compile(r".npy")
    mascara_test_pattern = re.compile(r".npy")
    
    #Almacenar los arrays cargados
    image_train_arrays = []
    image_test_arrays = []
    mascara_train_arrays = []
    mascara_test_arrays = []
    
    for file in files:
        if image_train_pattern.match(file):
            image_train_arrays.append(np.load(os.path.join(folder_path, file)))
        elif image_test_pattern.match(file):
            image_test_arrays.append(np.load(os.path.join(folder_path, file)))
        elif mascara_train_pattern.match(file):
            mascara_train_arrays.append(np.load(os.path.join(folder_path, file)))
        elif mascara_test_pattern.match(file):
            mascara_test_arrays.append(np.load(os.path.join(folder_path, file)))
    combined_images_train = np.concatenate(image_train_arrays, axis=0)
    combined_images_test = np.concatenate(image_test_arrays, axis=0)
    combined_mascara_train = np.concatenate(mascara_train_arrays, axis=0)
    combined_mascara_test = np.concatenate(mascara_test_arrays, axis=0)
    
    return combined_images_train, combined_images_test, combined_mascara_train, combined_mascara_test

folder_path = r"C:\Users\Usuari\Documents\TFM_Codigos\DATASET\Data-arrays"
images_train, images_test, mascara_train, mascara_test = load_and_combine_datasets(folder_path)

In [ ]:
mascara_train_category = to_categorical(mascara_train)
mascara_test_category = to_categorical(mascara_test)

Red FCN_ResNet50_Javi

In [ ]:
def FCN_ResNet50(input_shape, num_classes):
    # Usar ResNet50V2 como backbone
    base_model = tf.keras.applications.ResNet50V2(input_shape=input_shape, include_top=False, weights='imagenet')
    layer_names = [
        'conv2_block3_out',   
        'conv3_block4_out',   
        'conv4_block6_out',   
    ]
    layer_outputs = [base_model.get_layer(name).output for name in layer_names]
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=layer_outputs)
    down_stack.trainable = False
    x = layers.Conv2D(256, (1, 1), padding='same', activation='relu')(down_stack.output[-2])  # 8×8 
    x = layers.Dropout(0.5)(x)
    # Decodificador 
    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)  # 8×8 → 16×16
    skip_2 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(down_stack.outputs[1]) 
    x = Concatenate()([x, skip_2])
    x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 16×16 → 32×32
    skip_1 = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(down_stack.outputs[0])  
    x = Concatenate()([x, skip_1])
    x = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 32×32 → 64×64
    x = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 64×64 → 128×128
    # Salida final
    x = layers.Conv2D(num_classes, (1, 1), padding='same', activation='softmax')(x)  # (128×128×7)

    model = tf.keras.Model(inputs=down_stack.input, outputs=x)
    for i, layer_output in enumerate(down_stack.outputs):
        print(f"Salida {i}: {layer_output.shape}")
    return model

# Parámetros del modelo
input_shape = (128, 128, 3)  # Tamaño de las imágenes
num_classes = 7  # Número de clases

model = FCN_ResNet50(input_shape, num_classes)

Red MobileNetV2_Javi

In [ ]:
def FCN_MobileNetV2(input_shape, num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    
    layer_names = [
        'block_1_expand_relu',   
        'block_3_expand_relu',  
        'block_6_expand_relu',  
    ]
    layer_outputs = [base_model.get_layer(name).output for name in layer_names]
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=layer_outputs)
    down_stack.trainable = False
    x = layers.Conv2D(256, (3, 3), strides=2, padding='same', activation='relu')(down_stack.output[-1])  # 8×8 
    x = layers.Dropout(0.5)(x)
    # Decodificador
    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)  # 8×8 → 16×16
    x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 16×16 → 32×32
    x = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 32×32 → 64×64
    x = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)   # 64×64 → 128×128
    # Salida final
    x = layers.Conv2D(num_classes, (1, 1), padding='same', activation='softmax')(x)  # (128×128×7) Volvemos a la forma original
    # Crear el modelo final
    model = tf.keras.Model(inputs=down_stack.input, outputs=x)
    for i, layer_output in enumerate(down_stack.outputs):
        print(f"Salida {i}: {layer_output.shape}")
    return model


input_shape = (128, 128, 3)  # Tamaño de las imágenes
num_classes = 7  # Número de clases


model = FCN_MobileNetV2(input_shape, num_classes)

In [ ]:
#Compilar el modelo
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=CategoricalCrossentropy(),
    metrics=[MeanIoU(num_classes=num_classes), CategoricalAccuracy(), Precision(), Recall(), AUC()]
)
# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.1, patience=5)
]
# Entrenar el modelo
history = model.fit(
    x=images_train,
    y=mascara_train_category,
    batch_size=16,  # Aumentar el batch size
    epochs=35,
    validation_data=(images_test, mascara_test_category),
    callbacks=callbacks,
    shuffle=True,
    validation_split=0.1,
    verbose=1
)

Guardar el modelo

In [ ]:
#model.save('FCN_rESnEt50_v0.keras')
#model.save('FCN_MobileNet_v0.keras')

Analisis de los resultados + gráficas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

# Número de imágenes ráster a mostrar
num_samples = 10
indices = np.random.choice(len(images_test), num_samples, replace=False)

plt.figure(figsize=(12, num_samples * 3))

for i, idx in enumerate(indices):
    img = images_test[idx]
    true_mask = mascara_test[idx]  # Máscara real

    # Predecir la máscara
    pred_mask = model.predict(img[np.newaxis, ...])  
    pred_mask = np.argmax(pred_mask, axis=-1)[0] 

    # Mostrar la imagen original
    plt.subplot(num_samples, 3, i * 3 + 1)
    plt.imshow(img)
    plt.title("Imagen original")
    plt.axis("off")

    
    plt.subplot(num_samples, 3, i * 3 + 2)
    plt.imshow(true_mask, cmap=cmap, vmin=0, vmax=len(palette) - 1)  
    plt.title("Ground truth")
    plt.axis("off")

    plt.subplot(num_samples, 3, i * 3 + 3)
    plt.imshow(pred_mask, cmap=cmap, vmin=0, vmax=len(palette) - 1) 
    plt.title("Predicción")
    plt.axis("off")

plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(2, 5))
ax.legend(**legend)
ax.axis("off")
plt.show()


In [ ]:
# Obtener los valores del historial
loss = history.history['loss']
val_loss = history.history['val_loss']
iou = history.history['mean_io_u']  
val_iou = history.history['val_mean_io_u']

epochs = range(1, len(loss) + 1)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'bo-', label='Entrenamiento')
plt.plot(epochs, val_loss, 'ro-', label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
y_pred = model.predict(images_test)  
y_pred = np.argmax(y_pred, axis=-1)  
y_true = np.argmax(mascara_test_category, axis=-1)  
y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()

# Calcular precisión, recall y F1-score por clase
precision = precision_score(y_true_flat, y_pred_flat, average=None)  
recall = recall_score(y_true_flat, y_pred_flat, average=None) 
f1 = f1_score(y_true_flat, y_pred_flat, average=None) 
for i, f1_value in enumerate(f1):
    print(f"Clase {i}: F1-score = {f1_value:.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = model.predict(images_test)  
y_pred = np.argmax(y_pred, axis=-1)  
y_true = np.argmax(mascara_test_category, axis=-1)  
y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()

# Calcular la matriz de confusión
cm = confusion_matrix(y_true_flat, y_pred_flat, labels=np.arange(num_classes))

# Crear un gráfico con la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.show()

In [ ]:
# Normalizar la matriz de confusión (entre 0 y 1)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=labels, yticklabels=labels, cbar_kws={'label': 'Frecuencia Relativa'})
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
import pandas as pd
history_dict = history.history
# Extraer las métricas
epochs = range(1, len(history_dict['loss']) + 1)
loss = history_dict['loss']
val_loss = history_dict['val_loss']
iou = history_dict.get('mean_io_u', None)
val_iou = history_dict.get('val_mean_io_u', None)  
data = {
    'Epoch': epochs,
    'Loss': loss,
    'Validation Loss': val_loss,
    'Mean IoU': iou,
    'Validation Mean IoU': val_iou
}
df = pd.DataFrame(data)
df.to_csv('training_metrics.csv', index=False)
f1_scores = pd.DataFrame({
    'Class': labels,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
})

# Guardar los F1 Scores en un archivo CSV
f1_scores.to_csv('f1_scores.csv', index=False)

In [ ]:
#Gráficas del rendimiento del modelo durante aprendizaje
history = pd.DataFrame(history.history)
print(history)

plt.figure(figsize = (10, 8))
plt.plot(range(len(history['categorical_accuracy'].values.tolist())), history['categorical_accuracy'].values.tolist(), label = 'Train_Accuracy')
plt.plot(range(len(history['loss'].values.tolist())), history['loss'].values.tolist(), label = 'Train_Loss')
plt.plot(range(len(history['val_categorical_accuracy'].values.tolist())), history['val_categorical_accuracy'].values.tolist(), label = 'Test_Accuracy')
plt.plot(range(len(history['val_loss'].values.tolist())), history['val_loss'].values.tolist(), label = 'Test_Loss')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.legend()
plt.show()